# How to make your Python programs faster?
### by Dhruv Baldawa

## Disclaimer

```
This talk was previously named "Competition Programming and Problem solving using Python".
There might still be some remnants from that talk but I have updated it to be
more up-to-date and add other relevant information

# @TODO(DB): fix this description, it sucks!
```

## Why do we run into problems?

 - Big dataset
 - Great complexity
 - Python interpreter limitations
 - All of the above

## What this talk covers?

 - Memory management
   - lists v/s iterables
   - itertools _(my favorite part)_
 - Time management
   - correct data structures
   - approach
 - New advancements
   - asyncio
   - multiprocessing
   - numba / numpy and the likes
   - pyjit
   - Other interpreters
   - Other compilers
 - Profiling

## Memory management

### Why is important?

 - Most of the competitions require you store and compute results from large datasets.	
 - Running out of memory limit is pretty common.
 - There is "always" a way to improve memory consumption of your program.
 - You can always throw more hardware, but fixing it in the software just feels nice ;).

### Common problems

 - **MemoryError**[1]
   - when you run out of memory but still you can get out of it (by deleting few objects)
 - **OverflowError**[1]
   - when the results of an arithmetic operation is too large to be represented

### List forms

 - better to store and re-use results of computations	
 - use if you want to perform list operations, where you need to store all the items	
 - consumes	much more memory
 - only prefer when a real list cannot be avoided
 - all the elements are generated and initialized at once

### Iterables

 - lazy and on-demand generation of values
 - very low memory consumption
 - very useful when you only need to work with the value at hand
 - **Cons**:
   - cannot step backwards 
   - cannot or jump forwards
 - they are scalable and memory-friendly and used where real lists are not required

In [ ]:
BIG_NUMBER = 10**6

## range v/s xrange

#### range

In [ ]:
%%timeit
# range
sum = 0
l = range(BIG_NUMBER)
for x in l:
    sum += x

In [ ]:
# Access Operations
print l[-1] # Pass

# Assignment operation
l[10] = -1 # Pass
print l[10]

#### xrange

In [ ]:
%%timeit
# xrange
sum = 0
l = xrange(BIG_NUMBER)
for x in l:
    sum += x

In [ ]:
# Access Operations
print l[-1] # Pass

# Assignment operation
l[10] = -1 # FAILS !!
print l[10]

 - In Python 3, the default behavior for `range` is like `xrange` in Python 2
 - To get `range`-like behavior in Python 3, use `list(range(x, y))` instead

## List Comprehensions v/s Generator Expressions

### List Comprehensions

**Syntax**: `[expression for variables in iterable if condition]`

In [ ]:
%%timeit
# Using loops
square_list = []
for x in range(BIG_NUMBER):
    square_list.append(x**2)

In [ ]:
%%timeit
# List Comprehensions
square_list = [x**2 for x in range(BIG_NUMBER)]

In [ ]:
print square_list[:10]  # Works
print square_list[4]    # Works


### Generator Expressions

**Syntax**: `(expression for variables in iterable if condition)`

In [ ]:
%%timeit
# Generator Expressions
square_list = (x**2 for x in range(BIG_NUMBER))

In [ ]:
print square_list.next()

## Functions v/s Generators

#### Functions

```
def function(params):
    ....
    ....
    return something
```

In [ ]:
import math
def get_me_factorials_till(n):
    '''Returns a list of factorials from 1 to n'''
    return_list = []
    for x in range(1, n):
        return_list.append(math.factorial(x))
    return return_list

In [ ]:
factorials = get_me_factorials_till(10)
print factorials

#### Generators

```
def function(params):
    ...
    ...
    yield something
```

In [ ]:
import math
def generate_me_factorials_till(n):
    '''Generates factorials from 1 to n'''
    for x in range(1, n):
        yield math.factorial(x)

factorials = generate_me_factorials_till(10)

In [ ]:
print factorials

# Its an iterator
print factorials.next()

In [ ]:
# Playground
# So, it can be iterated on:
# Create a new generator, since the earlier one will be used up
factorials = generate_me_factorials_till(10)
for factorial in factorials:
    print factorial,

## Set and Dictionary Comprehensions

**Set Comprehension:**
`{value for value in expressions if conditions}`

**Dict Comprehensions:**
`{key: value for key, value in expressionss if conditions}`

In [ ]:
# Set comprehension
print {k for k in 'ABCDEFEDCBA'}

# Dictionary Comprehension
print {k: k+1 for k in range(10)}

## import itertools

 - itertools module provides a set of fast, memory efficient iterators
 - provides fast implementations for common jobs like product, permutations, combinations

<img src="itertools.png" />

In [4]:
# N-Queens Puzzle
# Reference: http://code.activestate.com/recipes/576647-eight-queens-six-lines/
from itertools import permutations

n = 4
cols = range(n)
for vec in permutations(cols):
    if (n == len(set(vec[i]+i for i in cols))
          == len(set(vec[i]-i for i in cols))):
        print vec

(1, 3, 0, 2)
(2, 0, 3, 1)


In [3]:
# run-length encoding
from itertools import groupby
for key, group in groupby("aaaabbccccddd"):
    print key, len(list(group)),

a 4 b 2 c 4 d 3


## Time optimizations

 - choosing the right data structure
 - choosing the right approach

    "Bad Programmers worry about code. Good programmers worry about data structures and their relationships"
    -- Linus Torvalds

 - **Don’t re-invent the wheel.** Use `tuple`, `list`, `dict`, `sets`, as they are coded in `C` and hence are _FAST_
 - for membership tests use `dict`/`set` [O(1)] instead of `list`/`tuple` [O(n)]
 - use `collections`
 - Queue operations like `pop()`, `insert()` are better in `collections.deque` [O(1)] than `list`[O(n) for insertion at the start]
 - use `bisect`, `heapq` for sorted lists


## import collections

 - `deque`
 - `Counter`
 - `OrderedDict`
 - `defaultdict`

#### deque

In [ ]:
import collections
# Creating a deque

d = collections.deque(['first', 'second', 'third', 'current last'])
print d
print '--'

In [ ]:
# right rotation
d.rotate()
print '>>> d.rotate()'
print d
print '--'

# left rotation
d.rotate(-1)
print '>>> d.rotate(-1)'
print d
print '--'

In [ ]:
# append from left side
d.appendleft("new first")
print '>>> d.appendleft("new first")'
print d
print '--'

# remove from the left side
d.popleft()
print '>>> d.popleft()'
print d
print '--'

In [ ]:
d.append("new last")
print '>>> d.append("new last")'
print d
print '--'

d.pop()
print '>>> d.pop()'
print d
print '--'

#### Counter

In [ ]:
# Playground
from collections import Counter
c = Counter(a=3, b=1)
d = Counter(a=1, b=2)

print c, d
print '--'

In [ ]:
# add two counters together:  c[x] + d[x]
print '>>> c + d'
print c + d
print '--'

# subtract (keeping only positive counts)
print '>>> c - d'
print c - d
print '--'

In [ ]:
# intersection:  min(c[x], d[x])
print '>>> c & d'
print c & d
print '--'

# union:  max(c[x], d[x])
print '>>> c | d'
print c | d
print '--'

In [ ]:
text = "I saw Susie sitting in a shoe shine shop."

import re
c = Counter(re.split(r"\W+", text, flags=re.IGNORECASE))
print c
print '--'

new_text = "Where she sits she shines, and where she shines she sits."
print '>>> c.update(new_text)'
c.update(re.split(r"\W+", new_text, flags=re.IGNORECASE))
print c
print '--'

print '>>> c.most_common(3)'
print c.most_common(3)
print '--'

#### defaultdict

In [ ]:
from collections import defaultdict

# defaultdict takes a default_factory as an argument
# you can always subclass it to have it with your own
# default factory
default_dict = defaultdict(int)
print default_dict[10]

## Memoization

### Collatz Conjecture

```
The following iterative sequence is defined for the set of positive integers:

n ->  n/2 (n is even)
n -> 3n + 1 (n is odd)

Using the rule above and starting with 13, we generate the following sequence:
13 -> 40 -> 20 -> 10 -> 5 -> 16 -> 8 -> 4 -> 2 -> 1

Which starting number, under one million, produces the longest chain?
```

In [ ]:
# Iterative solution
def chain(i):
    '''Iterative approach to solve the chain problem'''
    c = [i]
    last_element = i

    while last_element > 1:
        if last_element % 2 == 0:
            last_element = last_element / 2
        else:
            last_element = 3 * last_element + 1
        c.append(last_element)
    return c

def the_simple_chain():
    start = 2
    limit = 1000000
    max_length = 1
    max_num = 1
    for i in xrange(start, limit):
        c = chain(i)
        if len(c) > max_length:
            max_num = i
            max_length = len(c)

    return max_num

%timeit the_simple_chain()
# takes around 37.7s

In [ ]:
# Recursive solution
def chain(n):
    """
    Recursive function for the above expression
    """
    if n == 1:
        return [1]
    else:
        if n % 2 == 0:
            return [n] + chain(n / 2)
        else:
            return [n] + chain(3 * n + 1)

def the_recursive_chain():
    start = 2
    limit = 1000000
    max_length = 1
    max_num = 1

    for i in xrange(limit, start, -1):
        chain_list = chain(i)

        if len(chain_list) > max_length:
            max_length = len(chain_list)
            max_num = i

    return max_num

%timeit the_recursive_chain()
# takes around 1m 53s

In [ ]:
# Recursive solution
cache = {}
def chain(n):
    """
    Memoized recursive function for the above expression
    """
    if n in cache:
        return cache[n]
    if n == 1:s
        cache[1] = [1]
        return cache[1]
    else:
        if n % 2 == 0:
            cache[n] = [n] + chain(n / 2)
            return cache[n]
        else:
            cache[n] = [n] + chain(3 * n + 1)
            return cache[n]

def the_recursive_chain():
    start = 2
    limit = 1000000
    max_length = 1
    max_num = 1

    for i in xrange(limit, start, -1):
        chain_list = chain(i)

        if len(chain_list) > max_length:
            max_length = len(chain_list)
            max_num = i

    return max_num

# Don't you dare run this cell!
# %timeit the_recursive_chain()

## Multiplication Problem

**Naive Implementation**

2 * 3 = 2 + 2 + 2

In [ ]:
# Naive Multiplication
def naive(a, b):
    x = a
    y = b
    z = 0
    while x > 0:
        z = z + y
        x = x - 1
    return z

%timeit naive(BIG_NUMBER, BIG_NUMBER)
# 1 loops, best of 3: 11.6 s per loop

#### Russian Peasant's Algorithm

In [ ]:
# Russian Peasant's Algorithm for Multiplication
def russian(a, b):
    x = a
    y = b
    z = 0
    while x > 0:
        if x % 2 == 1:
            z = z + y
        y = y << 1
        x = x >> 1
    return z

%timeit russian(BIG_NUMBER, BIG_NUMBER)
# 100000 loops, best of 3: 7.4 us per loop

In [ ]:
# Recursive Russian Peasant's Algorithm
def rec_russian(a, b):
    if a == 0:
        return 0
    if a % 2 == 0:
        return 2 * rec_russian(a/2, b)
    return b + 2 * rec_russian((a-1)/2, b)

%timeit rec_russian(BIG_NUMBER, BIG_NUMBER)
# 100000 loops, best of 3: 9.47 us per loop